In [44]:
import pandas as pd
import numpy as np

In [210]:
df = pd.read_csv('/Users/serdarevichar/Library/CloudStorage/GoogleDrive-serdarevichar@gmail.com/My Drive/fantasy-football-database.csv', index_col = 'Unnamed: 0')

temp1 = df[['Year','Week','Playoff Flag','Home Team','Home Score','Away Score']].copy()
temp2 = df[['Year','Week','Playoff Flag','Away Team','Away Score','Home Score']].copy()

temp1.rename(columns={'Home Team':'Team', 'Home Score':'Score', 'Away Score':'Opp Score'}, inplace=True)
temp2.rename(columns={'Away Team':'Team', 'Away Score':'Score', 'Home Score':'Opp Score'}, inplace=True)

df_converted = pd.concat([temp1,temp2], axis=0, ignore_index=True)
df_converted['Win'] = df_converted['Score'] > df_converted['Opp Score']
df_converted['Win'] = df_converted['Win'].map(int)
df_converted

df_converted.sort_values(['Year','Week'], inplace = True)
df_converted

,Year,Week,Playoff Flag,Team,Score,Opp Score,Win
0,2019,1,False,Justin,158.02,113.20,1
1,2019,1,False,Dante,107.92,126.14,0
2,2019,1,False,Michael,112.68,134.32,0
3,2019,1,False,Andrew,128.54,141.26,0
4,2019,1,False,Haris,135.80,101.82,1
...,...,...,...,...,...,...,...
495,2024,16,True,Andrew,117.28,134.34,0
991,2024,16,True,Zach,139.88,136.02,1
992,2024,16,True,Noah,134.34,117.28,1
496,2024,17,True,Zach,128.60,89.22,1


# Weekly Scoreboard Table





In [89]:
week = df.loc[(df['Year'] == 2019) & (df['Week'] == 1)]
week

,Year,Week,Playoff Flag,Home Team,Home Score,Home Projection,Away Team,Away Score,Away Projection
0,2019,1,False,Justin,158.02,115.55,The,113.20,121.01
1,2019,1,False,Dante,107.92,118.65,Zach,126.14,114.46
2,2019,1,False,Michael,112.68,118.23,McGwire,134.32,122.20
3,2019,1,False,Andrew,128.54,121.59,Carter,141.26,117.11
4,2019,1,False,Haris,135.80,119.44,Ethan,101.82,112.52
5,2019,1,False,Noah,153.86,125.17,Nathan,132.22,116.77


In [7]:
scoreboard = week[['Home Team','Home Score','Away Score','Away Team']].copy()
scoreboard

,Home Team,Home Score,Away Score,Away Team
0,Justin,158.02,113.20,The
1,Dante,107.92,126.14,Zach
2,Michael,112.68,134.32,McGwire
3,Andrew,128.54,141.26,Carter
4,Haris,135.80,101.82,Ethan
5,Noah,153.86,132.22,Nathan


# Season Standings Table

In [222]:
year = 2019
week = 4

temp = df_converted.loc[(df_converted['Year'] == year) & (df_converted['Week'] <= week)]

teams = temp['Team'].unique()
records = []

for team in teams:
    temp_team = temp.loc[temp['Team'] == team]
    wins = temp_team['Win'].sum()
    losses = week - wins
    record = f'{wins}-{losses}'

    pf = round(temp_team['Score'].sum(), 2)
    avg_pf = round(pf / week, 2)
    pa = round(temp_team['Opp Score'].sum(), 2)
    avg_margin = round((pf - pa) / week, 2)

    records.append([team, record, pf, pa, avg_pf, avg_margin])

records

[['Justin', '3-1', 515.7, 432.76, 128.93, 20.74],
 ['Dante', '2-2', 455.06, 449.84, 113.76, 1.31],
 ['Michael', '2-2', 490.94, 568.2, 122.74, -19.32],
 ['Andrew', '1-3', 540.02, 570.16, 135.0, -7.53],
 ['Haris', '2-2', 471.32, 478.12, 117.83, -1.7],
 ['Noah ', '2-2', 534.0, 469.4, 133.5, 16.15],
 ['The', '3-1', 462.34, 435.56, 115.58, 6.69],
 ['Zach', '1-3', 443.16, 460.52, 110.79, -4.34],
 ['McGwire', '2-2', 425.52, 448.94, 106.38, -5.86],
 ['Carter', '3-1', 460.76, 433.56, 115.19, 6.8],
 ['Ethan', '1-3', 410.36, 462.08, 102.59, -12.93],
 ['Nathan', '2-2', 514.72, 514.76, 128.68, -0.01]]

In [224]:
table = pd.DataFrame(records, columns=['Team','Record', 'Points For','Points Against','Avg Points For','Avg Margin'])
table.sort_values(['Record','Points For'], ascending=False, ignore_index=True)

,Team,Record,Points For,Points Against,Avg Points For,Avg Margin
0,Justin,3-1,515.70,432.76,128.93,20.74
1,The,3-1,462.34,435.56,115.58,6.69
2,Carter,3-1,460.76,433.56,115.19,6.80
3,Noah,2-2,534.00,469.40,133.50,16.15
4,Nathan,2-2,514.72,514.76,128.68,-0.01
5,Michael,2-2,490.94,568.20,122.74,-19.32
6,Haris,2-2,471.32,478.12,117.83,-1.70
7,Dante,2-2,455.06,449.84,113.76,1.31
8,McGwire,2-2,425.52,448.94,106.38,-5.86
9,Andrew,1-3,540.02,570.16,135.00,-7.53


# Luck Score Table




In [125]:
standings_data.head()

,Team,Score,Opp Score,Win
0,Justin,158.02,113.20,1
1,Dante,107.92,126.14,0
2,Michael,112.68,134.32,0
3,Andrew,128.54,141.26,0
4,Haris,135.80,101.82,1


In [156]:
mean = standings_data['Score'].mean()
std = standings_data['Score'].std() * 0.5

print(mean,std)

119.09870370370369 11.986959368542161


In [76]:
def opp_luck_score(score, mean=None, std=None):
    return int(((mean - score) / np.absolute(mean - score)) * np.floor(np.absolute(mean - score) / std))

In [157]:
opp_luck_score(100,mean=mean,std=std)

1

In [158]:
standings_data['Opp Luck Score'] = standings_data[['Opp Score']].map(opp_luck_score, mean=mean, std=std)
standings_data

,Team,Score,Opp Score,Win,Opp Luck Score,Your Luck Score
0,Justin,158.02,113.20,1,0,0
1,Dante,107.92,126.14,0,0,0
2,Michael,112.68,134.32,0,-1,0
3,Andrew,128.54,141.26,0,-1,0
4,Haris,135.80,101.82,1,1,0
...,...,...,...,...,...,...
103,Michael,108.56,109.78,0,0,0
104,Dante,117.50,133.12,0,-1,0
105,Justin,115.30,122.36,0,0,0
106,Zach,115.80,100.66,1,1,0


In [139]:
def your_luck_score(score, result, mean=None, std=None):
    xi = lambda x: np.floor((mean + x) / (2*mean)) 
    return int(np.floor(np.absolute(mean - score) / std) * (result * xi(2*mean - score) - (1-result) * xi(score)))

In [141]:
your_luck_score(175,0, mean=mean, std=std)

-2

In [162]:
yls = [your_luck_score(score, result, mean=mean, std=std) for score, result in standings_data[['Score','Win']].values]
standings_data['Your Luck Score'] = yls
standings_data.sort_values('Score', ascending=False).head(10)

,Team,Score,Opp Score,Win,Opp Luck Score,Your Luck Score
30,The,183.40,122.04,1,0,0
70,Noah,181.58,108.72,1,0,0
83,Michael,172.26,172.16,1,-4,0
29,Dante,172.16,172.26,0,-4,-4
28,Justin,164.64,101.92,1,1,0
25,Carter,163.82,114.76,1,0,0
77,Michael,162.32,146.48,1,-2,0
48,Andrew,161.64,137.32,1,-1,0
0,Justin,158.02,113.20,1,0,0
43,Andrew,155.50,108.74,1,0,0


In [164]:
def close_luck_score(score,opp_score):
    if (score - opp_score) < 3 and (score - opp_score) > 0:
        return 1
    elif (score - opp_score) < 0 and (score - opp_score) > -3:
        return -1
    else:
        return 0    

In [171]:
cls = [close_luck_score(score, opp_score) for score, opp_score in standings_data[['Score','Opp Score']].values]
standings_data['Close Luck Score'] = cls
standings_data['Luck Score'] = standings_data[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)


# Champions Table

In [211]:
champions = df_converted.loc[(df_converted['Week'] == 17) & (df_converted['Win'] == 1), ['Year','Team']].values
champions

array([[2019, 'Dante'],
       [2020, 'Ethan'],
       [2021, 'Andrew'],
       [2022, 'Dante'],
       [2023, 'Haris'],
       [2024, 'Zach']], dtype=object)

In [226]:
data = []
for year, team in champions:
    temp = df_converted.loc[(df_converted['Year'] == year) & (df_converted['Team'] == team) & (df_converted['Playoff Flag'] == False)]
    wins = temp['Win'].sum()
    losses = len(temp) - wins
    record = f'{wins}-{losses}'

    pf = round(temp['Score'].sum(), 2)
    avg_pf = round(pf / len(temp), 2)
    pa = round(temp['Opp Score'].sum(), 2)
    avg_margin = round((pf - pa) / len(temp), 2)

    data.append([year, team, record, pf, avg_pf, avg_margin])
    
pd.DataFrame(data, columns=['Year','Team','Record','Points For','Avg Points For','Avg Margin'])

,Year,Team,Record,Points For,Avg Points For,Avg Margin
0,2019,Dante,8-5,1612.06,124.00,9.25
1,2020,Ethan,11-2,1639.56,126.12,22.44
2,2021,Andrew,10-4,1901.54,135.82,10.45
3,2022,Dante,10-4,1747.00,124.79,12.49
4,2023,Haris,12-2,1801.34,128.67,24.65
5,2024,Zach,8-6,1762.04,125.86,6.56


In [232]:
years = np.arange(2019,2025)
for year in years:
    if year < 2021:
        weeks = np.arange(1,14)
    else:
        weeks = np.arange(1,15)
    print(weeks)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
